In [21]:
import pandas as pd
import numpy as np
from IPython.display import Image

# Import Y

In [2]:
Y=pd.read_csv('comm-at-least-4-Y.csv',header=0,index_col=0)
Y=Y.sort_index(axis=0, level=None, ascending=True)
Y=Y.fillna(value=0)


# Import and Transform X

In [3]:
likeall=pd.read_csv("./comm-at-least-4-engage.txt",header=None, names=['time','pageid','uid'],sep=' ')
likeall['count']=1

In [4]:
new_metric=likeall.pivot_table(index='uid',columns='pageid',values='count',aggfunc=np.sum)

In [33]:
#Save metric
new_metric.to_csv(path_or_buf='./comm-at-least-4-engage-metric.csv')

In [5]:
new_metric=new_metric.fillna(value=0)

# Feature Selection: Variance

In [6]:
from sklearn.feature_selection import VarianceThreshold
selector = VarianceThreshold(threshold=0.09)
metric_variance=selector.fit_transform(new_metric)

In [7]:
print new_metric.shape
print metric_variance.shape

(2015, 895)
(2015L, 67L)


# Split Test Set

In [395]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(metric_variance, Y['Blue'], test_size=200,random_state=42)
y_train=np.asarray(y_train,dtype="float64")
y_test=np.asarray(y_test,dtype="float64")

In [396]:
print X_test.shape, y_test.shape,X_train.shape,y_train.shape

(200L, 67L) (200L,) (1815L, 67L) (1815L,)


In [10]:
X_selected=X_train
Y_label=np.asarray(y_train, dtype="float64")

In [208]:
y_train.sum()

51.0

In [12]:
X_selected.shape

(1515L, 67L)

## Under-sampling

In [985]:
list=np.where(Y_label==0)

In [986]:
len(list[0])

1742

In [987]:
del_index=np.random.choice(list[0],size=(1748-73), replace=False)

In [988]:
Y_label=np.delete(Y_label,del_index, None)

In [989]:
X_selected=np.delete(X_selected,del_index, 0)

In [990]:
print Y_label.shape
print X_selected.shape

(140L,)
(140L, 67L)


## Over-sampling SMOTE

In [336]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(kind='regular',ratio=0.1)
X_train, y_train = sm.fit_sample(X_train, y_train)

In [337]:
print X_train.shape
print len(y_train[y_train==1])

(1919L, 67L)
174


# PCA

In [861]:
from sklearn.decomposition import PCA
pca = PCA(n_components=10)
#new_metric=new_metric.fillna(value=0)
metric_pca=pca.fit_transform(X_train)
#metric_pca=pca.fit_transform(pre_metric)

In [146]:
pca.explained_variance_ratio_.sum()

0.58833931652814542

In [862]:
print metric_pca.shape

(1015L, 10L)


# NMF

In [874]:
from sklearn.decomposition import NMF
model = NMF(n_components=10, init='random',alpha=0.01,max_iter=500,tol=1e-4,random_state=0)
model=model.fit(X_train)
metric_nmf=model.transform(X_train)

In [875]:
X_test=model.transform(X_test)

In [35]:
#component-page
model.components_.shape
#user-component
metric_nmf.shape

(1815L, 10L)

###  Assign X_train

In [168]:
X_train=new_metric

In [113]:
X_train=metric_variance

In [863]:
X_train=metric_pca

In [867]:
X_train=metric_nmf

In [868]:
X_train.shape

(1015L, 10L)

## My F-beta score

Difference with default F1 score:
When deal with tricky metric like this
<pre>
|         |      |     True    |
|         |      |   0  |   1  |
|---------|------|------|------|
| Predict |   0  |   3  |   0  |
|         |   1  |   4  |   0  |
</pre>
No real positive samples makes the denominator and numeratore of F1 metric equal to 0. F1, recall, precision = 0/0.
Default F1 test are all equal to 0. But in my opinion: F1, recall, precision score are 0, 1, 0 respectively.

-------
And this:
          True
           0
Predict   0   3
In this case, I will set all of F1, recall, precision to 1. However, the default is 0.

--------
Besides, my version could return the cross table as above.

--------

Note about Beta:
![title](f1score.svg)
beta < 1 lends more weight to precision, while beta > 1 favors recall

In [93]:
def Score (PredictList,TrueList,beta):
    X=pd.concat([pd.DataFrame(PredictList),pd.DataFrame(TrueList)],axis=1, ignore_index=True)
    X['count']=1
    X.columns=['Predict','True','count']
    Acc=X.pivot_table(index='Predict',columns='True',values='count',aggfunc=np.sum)
    Acc=Acc.fillna(value=0)
    rate,f1,recall,precision=0,0,0,0
    b=beta**2
    if Acc.shape==(2,2):
        rate=rate+(float(Acc[0][0]+Acc[1][1])/Acc.sum().sum())
        if Acc[1][1]!=0:
            recall=float(Acc[1][1])/float(Acc[1][0]+Acc[1][1])
            precision=float(Acc[1][1])/float(Acc[0][1]+Acc[1][1])
            f1=(1+b) * (float(Acc[1][1])/float(Acc[0][1]+Acc[1][1])) * (float(Acc[1][1])/float(Acc[1][0]+Acc[1][1])) / ((b * float(Acc[1][1])/float(Acc[0][1]+Acc[1][1])) + (float(Acc[1][1])/float(Acc[1][0]+Acc[1][1])))
    elif Acc.shape==(1,2):
        if Acc.index[0]==0:
            rate=rate+(float(Acc[0][0])/Acc.sum().sum())
        else:
            rate=float(Acc[0][1])/Acc.sum().sum()
            recall=1
            precision=float(Acc[0][1])/Acc.sum().sum()
            f1=(1+b) * ((float(Acc[0][1])/Acc.sum().sum()) * 1) / ((b * float(Acc[0][1])/Acc.sum().sum()) + 1)
    elif Acc.shape==(2,1):
        if Acc.columns==1:
            rate=float(Acc.iloc[1])/Acc.sum().sum()
            recall=float(Acc.iloc[1])/Acc.sum().sum()
            precision=1
            f1=(1+b) * float(Acc.iloc[1])/Acc.sum().sum()/(float(Acc.iloc[1])/Acc.sum().sum()+ b)
        else:
            rate=float(Acc.iloc[0])/Acc.sum().sum()
            recall=1
    else: #Acc.shape=(1,1)
        rate=1
        recall=1
        precision=1
        f1=1
    return Acc,rate,f1,recall,precision

## Feature Selection: Correlation p value

In [37]:
from scipy import stats
from sklearn.linear_model import LogisticRegression
LogReg=LogisticRegression()
LogReg.fit(X_selected, Y_label)
sse = np.sum((LogReg.predict(X_selected) - Y_label) ** 2, axis=0) / float(X_selected.shape[0] - X_selected.shape[1])
se = np.array([
        np.sqrt(np.diagonal(sse * np.linalg.inv(np.dot(X_selected.T, X_selected))))
    ])
t = LogReg.coef_ / se
p = 2 * (1 - stats.t.cdf(np.abs(t), Y_label.shape[0] - X_selected.shape[1]))
np.set_printoptions(suppress=True)
print 'How many variables are significantly related to Y:',len(p[p<0.05])
print p

How many variables are significantly related to Y: 9
[[ 0.          0.          0.          0.08084395  0.00173334  0.          0.
   0.          0.00021892  0.        ]]


## Feature Selection: RFE

In [38]:
from sklearn.feature_selection import RFE
selector = RFE(LogReg, 9, step=1)
selector = selector.fit(X_selected, Y_label)
selector.support_

array([ True,  True,  True, False,  True,  True,  True,  True,  True,  True], dtype=bool)

In [39]:
X_selected=X_selected[:,selector.support_]
X_selected.shape

(1815L, 9L)

## Feature Selection: Chi2

In [45]:
# Chi Square
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [1020]:
#p value
len(chi2(X_selected, Y_label)[1][chi2(X_selected, Y_label)[1]<0.05])

46

In [1021]:
SKB = SelectKBest(chi2, k=46).fit(X_selected, Y_label)

In [1022]:
X_selected=SKB.transform(X_selected)

In [1023]:
X_test=SKB.transform(X_test)

# Model Comparison

In [413]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
#cost-sensitivity
from costcla.models import BayesMinimumRiskClassifier
#cross-validation
from sklearn.cross_validation import KFold
k=10
kf = KFold(1815, n_folds=k,shuffle=True)

classifiers = {"MulBayes": {"f": MultinomialNB()},
               "RandomForest": {"f": RandomForestClassifier()},
               "DecisionTree": {"f": DecisionTreeClassifier()},
               "LogReg": {"f": LogisticRegression()}}

beta=1
for model in classifiers.keys():
    recall,rate,precision,f1 = 0.0,0.0,0.0,0.0
    for train_index, test_index in kf:
        # Fit
        classifiers[model]["f"].fit(X_train[train_index], y_train[train_index])
        # Predict
        Acc,rate_fold,f1_fold,recall_fold,precision_fold=Score(classifiers[model]["f"].predict(X_train[test_index]),y_train[test_index],beta)
        rate+=rate_fold
        f1+=f1_fold
        recall+=recall_fold
        precision+=precision_fold
    rate=float(rate)/k
    recall=float(recall)/k
    precision=float(precision)/k
    f1=float(f1)/k
    classifiers[model]["Scores"]={"accuracy":rate, "f-beta":f1, "recall":recall, "precision":precision}
    classifiers[model]["predict"] = classifiers[model]["f"].predict(X_train)
    classifiers[model]["predict_prob"] = classifiers[model]["f"].predict_proba(X_train)
        
# Evaluate the performance
results = pd.DataFrame(columns=classifiers[classifiers.keys()[0]]["Scores"].keys())

for model in classifiers.keys():
    results.loc[model] = classifiers[model]["Scores"].values()

print results[["accuracy","f-beta","recall","precision"]]

                  accuracy    f-beta    recall  precision
LogReg            0.958718  0.015385  0.010000   0.033333
RandomForest      0.954301  0.062626  0.054286   0.075000
DecisionTree      0.927287  0.069098  0.066508   0.076349
MulBayes          0.874376  0.270337  0.581934   0.179876
LogReg-BMR        0.917904  0.298677  0.481667   0.222747
RandomForest-BMR  0.981817  0.778637  0.875476   0.706209
DecisionTree-BMR  0.984579  0.801067  0.831190   0.788611
MulBayes-BMR      0.896430  0.239180  0.443095   0.167894


# Cost sensitivity

WARNING: The "costcla" is not reliable. The results look weird and different with my maunual calculation. Unsure whose problem is this, mine or costcla's.

In [ ]:

#add cost-sensitivity model
cost=1 #switch
# cost_mat[C_FP,C_FN,C_TP,C_TN]
FP=10
FN=10
cost_mat=np.zeros((len(y_train),4))
for i in range(len(y_train)):
    cost_mat[i]=[FP,FN*X_train[i].sum(),0.0,0.0]
if cost==1:
    kf = KFold(1815, n_folds=k,shuffle=True)
    for model in classifiers.keys():
        classifiers[model+"-BMR"] = {"f": BayesMinimumRiskClassifier()}
        recall,rate,precision,f1 = 0.0,0.0,0.0,0.0
        for train_index, test_index in kf:
            #fit
            classifiers[model+"-BMR"]["f"].fit(y_train[train_index], classifiers[model]["predict_prob"][train_index])
            # Predict
            predict = classifiers[model+"-BMR"]["f"].predict(classifiers[model]["predict_prob"][test_index], cost_mat[test_index])
            Acc,rate_fold,f1_fold,recall_fold,precision_fold=Score(predict,y_train[test_index],beta)
            ''''if model=='DecisionTree':
                print Acc
                print recall_fold,precision_fold
                print '------------'''
            rate+=rate_fold
            f1+=f1_fold
            recall+=recall_fold
            precision+=precision_fold
        rate=float(rate)/k
        recall=float(recall)/k
        precision=float(precision)/k
        f1=float(f1)/k
        classifiers[model+"-BMR"]["Scores"]={"accuracy":rate, "f-beta":f1, "recall":recall, "precision":precision}
        results.loc[model+"-BMR"] = 0
        results.loc[model+"-BMR"] = classifiers[model+"-BMR"]["Scores"].values()

print results[["accuracy","f-beta","recall","precision"]]

# Test

In [415]:
results = pd.DataFrame(columns=["accuracy","f-beta","recall","precision"])
for model in classifiers.keys():
    if "BMR" not in model:
        Acc,rate,f1,recall,precision=Score(classifiers[model]["f"].predict(X_test),y_test,beta)
        results.loc[model]=rate,f1,recall,precision
        cost_mat=np.zeros((len(y_test),4))
        for i in range(len(y_test)):
            cost_mat[i]=[FP*X_test[i].sum(),FN*X_test[i].sum(),0.0,0.0]
        predict = classifiers[model+"-BMR"]["f"].predict(classifiers[model]["f"].predict_proba(X_test), cost_mat)
        Acc,rate,f1,recall,precision=Score(predict,y_test,beta)
        print model,'-BMR'
        print Acc
        results.loc[model+"-BMR"]=rate,f1,recall,precision

print results

RandomForest -BMR
True     0.0  1.0
Predict          
0.0      188    6
1.0        5    1
LogReg -BMR
True     0.0  1.0
Predict          
0.0      193    7
DecisionTree -BMR
True     0.0  1.0
Predict          
0.0      185    5
1.0        8    2
MulBayes -BMR
True     0.0  1.0
Predict          
0.0      193    7
                  accuracy    f-beta    recall  precision
RandomForest         0.950  0.166667  0.142857   0.200000
RandomForest-BMR     0.945  0.153846  0.142857   0.166667
LogReg               0.965  0.000000  0.000000   0.000000
LogReg-BMR           0.965  0.000000  0.000000   0.000000
DecisionTree         0.935  0.235294  0.285714   0.200000
DecisionTree-BMR     0.935  0.235294  0.285714   0.200000
MulBayes             0.855  0.216216  0.571429   0.133333
MulBayes-BMR         0.965  0.000000  0.000000   0.000000
